In [14]:
import os
import sys
import warnings
import time
from pynq import PL
from pynq import Overlay

BF = os.path.join("./data/system_wrapper.bit")

# Download the bitfile

if not os.path.exists(BF):
    print ("%s Doesn't exist!" % BF)
else:
    print ("Found bit file!")
    
    ol = Overlay(BF)
    ol.download()

    print ("Bitfile downloaded!")

Found bit file!
Bitfile downloaded!


In [17]:
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir) 

from drivers.video_mixer import VideoMixer
from drivers.dynamic_clock import DynamicClock
from drivers.timing_controller import TimingController
from drivers.test_pattern_generator import TestPatternGenerator
from drivers.test_pattern_generator import TestPatternID
from drivers.axi_graphics import AXIGraphics

DEBUG   = False

print ("Starting Video")

print ("Configuring Timing Controller")
tc = TimingController("video/timing_generator")
tc.reset()
while (not tc.is_reset_done()):
	print (".")

WIDTH   = tc.get_generator_width()
HEIGHT  = tc.get_generator_height()
print ("Image Size (Retrieved from Timing Controller): %d x %d" % (WIDTH, HEIGHT))
print ("Configuring Test Pattern Generator")
tpg = TestPatternGenerator("video/v_tpg_0", debug = DEBUG)
tpg.set_image_size(WIDTH, HEIGHT)
tpg.set_color_format_to_rgb()
#tpg.set_color_bar_test_pattern()
#tpg.set_test_pattern(TestPatternID.SOLID_RED)
#tpg.set_test_pattern(TestPatternID.SOLID_WHITE)
tpg.set_test_pattern(TestPatternID.SOLID_BLACK)
#tpg.set_test_pattern(TestPatternID.COLOR_BARS)
tpg.start()

SUB_WIN_WIDTH = 640
SUB_WIN_HEIGHT = 480
#SUB_WIN_WIDTH = 16
#SUB_WIN_HEIGHT = 4
#SUB_WIN_WIDTH = WIDTH
#SUB_WIN_HEIGHT = HEIGHT

print ("Configuring Video Mixer")
vm = VideoMixer("video/v_mix_0", WIDTH, HEIGHT)
#vm.configure_layer(0, 0, 0, WIDTH, HEIGHT)
vm.configure_layer(1, 0, 0, SUB_WIN_WIDTH, SUB_WIN_HEIGHT)
vm.enable_layer(1, True)
vm.configure_master_layer(WIDTH, HEIGHT)
#vm.enable_overlay_layer(True)

if DEBUG: print ("Video Mixer Control Register Before Enable: 0x%08X" % vm.get_control())
vm.start()

if DEBUG: print ("Video Mixer Control Register After Enable: 0x%08X" % vm.get_control())
print ("Enable Master Layer (Test Pattern) Output")
vm.enable_master_layer(True)
if DEBUG: print ("VM Settings:")
if DEBUG: print ("  x pos:   %d" % vm.get_layer_x(0))
if DEBUG: print ("  y pos:   %d" % vm.get_layer_y(0))
if DEBUG: print ("  width:   %d" % vm.get_layer_width(0))
if DEBUG: print ("  height:  %d" % vm.get_layer_height(0))
if DEBUG: print ("  scale:   %d" % vm.get_layer_scale(0))
if DEBUG: print ("  alpha:   %d" % vm.get_layer_alpha(0))
if DEBUG: print ("  stride:  %d" % vm.get_layer_stride(0))
if DEBUG: print ("  p1 Addr: %d" % vm.get_layer_plane1_addr(0))
if DEBUG: print ("  p2 Addr: %d" % vm.get_layer_plane2_addr(0))

if DEBUG: print ("Layer Settings: 0x%08X" % vm.get_layer_enable_reg())

g0 = ol.low_speed.axi_gpio_0
g0.write(0x0C, 0x01)
tc.enable(gen_enable = True, use_gen_src = True)
if DEBUG: print ("TC Control Register: 0x%08X" % tc.get_control_reg())

print ("Interfacing with AXI Graphics Controller")
if DEBUG: print ("%s" % str(ol.ip_dict["video/axi_graphics_0"]))
ag = AXIGraphics("video/axi_graphics_0", debug = DEBUG)
#print ("AXI Graphics Started: %s" % str(ol.ip_dict["video/axi_graphics"]))
#time.sleep(0.1)
#time.sleep(5)
if DEBUG: print ("AXI Graphics Control Register: 0x%08X" % ag.get_control())
#print ("AXI Graphics: 0x%08X" % ag.get_version())
ag.set_width(SUB_WIN_WIDTH)
ag.set_height(SUB_WIN_HEIGHT)
print ("Size: %d x %d" % (ag.get_width(), ag.get_height()))
#ag.set_mode(0)  # Black
#ag.set_mode(1)  # White
#ag.set_mode(2)  # Red
#ag.set_mode(3)  # Green
#ag.set_mode(4)  # Blue
ag.set_mode(5)  # Color Bars
#ag.set_mode(6)  # Block
#ag.set_mode(7)  # Ramp
ag.set_alpha(0xFF)
ag.set_ref0_xy(100, 100)
ag.set_ref1_xy(200, 200)
ag.set_interval(100)
ag.enable_rgba_format(True)
#ag.enable_rgba_format(False)
ag.enable(True)
if DEBUG: print ("AXI Graphics Control Register: 0x%08X" % ag.get_control())
#print ("Sleeping for 5 seconds")
#time.sleep(5)


Starting Video
Configuring Timing Controller
Image Size (Retrieved from Timing Controller): 1920 x 1080
Configuring Test Pattern Generator
Configuring Video Mixer
Enable Master Layer (Test Pattern) Output
Interfacing with AXI Graphics Controller: 

Size: 640 x 480
